In [1]:
from bertopic import BERTopic
from scipy.cluster import hierarchy as sch
import pandas as pd
import ast
from sklearn.metrics.pairwise import cosine_distances
from tqdm import tqdm
import numpy as np
from umap import UMAP
from hdbscan import HDBSCAN
from sentence_transformers import SentenceTransformer
import openai
import re
import sys
sys.path.append('../../')
import json
from utility.llm_utility import *

In [2]:
df = pd.read_parquet('../../data/avocado_train.parquet',engine='fastparquet')

#### Split DF into chunks of max. 50.000

In [3]:
# Number of rows per subset
chunk_size = 10000

# List to store subsets
df_subsets = [df.iloc[i:i + chunk_size] for i in range(0, len(df), chunk_size)]

# Access each subset
for i, subset in enumerate(df_subsets):
    print(f"Subset {i + 1}: {len(subset)} rows")

Subset 1: 10000 rows
Subset 2: 10000 rows
Subset 3: 10000 rows
Subset 4: 10000 rows
Subset 5: 10000 rows
Subset 6: 10000 rows
Subset 7: 10000 rows
Subset 8: 10000 rows
Subset 9: 10000 rows
Subset 10: 10000 rows
Subset 11: 10000 rows
Subset 12: 10000 rows
Subset 13: 10000 rows
Subset 14: 10000 rows
Subset 15: 10000 rows
Subset 16: 10000 rows
Subset 17: 10000 rows
Subset 18: 10000 rows
Subset 19: 10000 rows
Subset 20: 10000 rows
Subset 21: 10000 rows
Subset 22: 10000 rows
Subset 23: 10000 rows
Subset 24: 10000 rows
Subset 25: 10000 rows
Subset 26: 10000 rows
Subset 27: 10000 rows
Subset 28: 10000 rows
Subset 29: 10000 rows
Subset 30: 8988 rows


#### Create JSONL Files

In [6]:
for i, df_subset in enumerate(df_subsets):
    batch_filepath = f'../../data/llm_batches/signature_removal/batch_{i}.jsonl'
    batch_file = create_json_batch(
            df_subset=df_subset,
            system_prompt="You are a concise assistant whose answer is merely the desired output to a task given by the user.",
            userprompt="The following email-text I provide to you in triple-quotation marks might contain a signature. I want you to return the original email-text but exclude the signature.:'''{email}'''",
            )

    # Write to JSONL file
    with open(batch_filepath, "w") as f:
        for line in batch_file:
            # Serialize each object as a JSON string and write it to the file
            f.write(json.dumps(line) + "\n")

In [5]:
import tiktoken

# Choose the model you are using
model = "gpt-4o-mini"  # or "gpt-3.5-turbo", "davinci", etc.

# Load the tokenizer for the chosen model
encoding = tiktoken.encoding_for_model(model)

# Load the content of your batch file
with open("../../data/llm_batches/signature_removal/batch_0.jsonl", "r", encoding="utf-8") as file:
    content = file.read()

# Count the tokens
tokens = encoding.encode(content)
token_count = len(tokens)

print(f"Number of tokens in the file: {token_count}")


Number of tokens in the file: 1932005
